In [1]:
import boto3
import io
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

# export to S-3
import awswrangler as wr

# Imputation
from sklearn.impute import SimpleImputer


plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#!pip install awswrangler

In [2]:
pd.set_option('display.max_rows', 200)

### Load a consolidated and expanded file from S-3

In [3]:
session = boto3.Session()
s3 = session.client('s3')

In [4]:
bucket_name = 'ppsg-collections'
file_name = '2021-06-01/consolidated/random_data/consolidated_sample_expanded.parquet'

In [5]:
%%time
obj = s3.get_object(Bucket = bucket_name, Key = file_name)
df = pd.read_parquet(io.BytesIO(obj['Body'].read())) 

CPU times: user 1min 57s, sys: 6min 10s, total: 8min 8s
Wall time: 54.8 s


In [6]:
df.isnull().sum() / len(df)

loanid                              0.000000e+00
loanstatusid                        0.000000e+00
nextpaymentduedt                    0.000000e+00
rundt                               0.000000e+00
ndd                                 0.000000e+00
rundatenumber                       0.000000e+00
priormeperiod                       0.000000e+00
MEPeriod                            0.000000e+00
loanstatus                          0.000000e+00
priorstatus                         5.800617e-03
EscrowFlag                          0.000000e+00
currentescrowpayment                0.000000e+00
nextescrowpayment                   5.114234e-05
currentpipayment                    0.000000e+00
nextpipayment                       5.800617e-03
OpenLossDraftClaimAmount            0.000000e+00
upb                                 0.000000e+00
priorupb                            5.800617e-03
CurrentAmortTerm                    0.000000e+00
LastPaymentEffectiveDate            1.535907e-02
LastPaymentDt       

In [7]:
upb1=df[df['upb'].isnull()]

In [9]:
df.dtypes

loanid                                       int64
loanstatusid                                object
nextpaymentduedt                    datetime64[ns]
rundt                               datetime64[ns]
ndd                                        float64
rundatenumber                              float64
priormeperiod                              float64
MEPeriod                                   float64
loanstatus                                  object
priorstatus                                 object
EscrowFlag                                  object
currentescrowpayment                       float64
nextescrowpayment                          float64
currentpipayment                           float64
nextpipayment                              float64
OpenLossDraftClaimAmount                   float64
upb                                        float64
priorupb                                   float64
CurrentAmortTerm                           float64
LastPaymentEffectiveDate       

In [8]:
upb1.groupby(['nextpaymentduedt'])['loanid'].count()

Series([], Name: loanid, dtype: int64)

### Data Treatments 

In [8]:
def avg_payment_day(col):
    conditions  = [df[col] <= 5, (df[col] >5) & (df[col] <= 10),  (df[col]>=11) ]
    choices     = [ '1_5', '6_10' , '11_30']
    df[col + '_bucket'] = np.select(conditions, choices, default='missing')

#### Change Data Types

In [ ]:
# To Numeric
#num_cols = ['origbal','currentescrowpayment', 'nextescrowpayment', 'currentpipayment', 'nextpipayment', 
#        'OpenLossDraftClaimAmount','upb','priorupb'] 

#df[num_cols]  = df[num_cols].apply(pd.to_numeric)

In [9]:
# Datetime
dt_cols = ['PurchaseDt', 'nextLastPaymentDt', 'LastPaymentDt', 'LastPaymentEffectiveDate', 'DraftLastScheduledDt',
        'LoanModificationDt', 'acquisitiondt', 'mostrecentprokenpromisehistory', 'mostrecentcontact', 'NoteDt']

df[dt_cols] = df[dt_cols].apply(pd.to_datetime)

#### Remove Data with no UPB and no Purchase Date

In [10]:
# remove loans no upb
df = df.loc[df['upb']>1]

## remove loans that has no purchase date
df = df.dropna(axis=0, subset=['PurchaseDt'])

#### Date Treatments

In [11]:
## Date Treatments
df['rundt_day'] = df['rundt'].dt.day  
df['rundt_weekday'] = df['rundt'].dt.weekday
df['nextpaymentdue_day'] = df['nextpaymentduedt'].dt.day  
df['nextpayment_weekday'] = df['nextpaymentduedt'].dt.dayofweek

# get # of days in a month
df['days_in_month'] = df['nextpaymentduedt'].dt.daysinmonth

#get age
df['age'] = np.round((df['nextpaymentduedt'] - df['PurchaseDt'])/np.timedelta64(1, 'M'), 0).astype(int)

#### Numerical Features

##### FICO

In [12]:
# Create a flag for FICO = 200 population
df['FICO_200'] = np.where(df['FICOScore']==200, 1,0)
df['FICOScore'] = np.where(df['FICOScore'].isnull(),0,df['FICOScore'])
df['FICO_flag'] = np.where(df['FICOScore']==0,1,0)

#### Mean Imputation

In [13]:
#cols = df.select_dtypes(np.floating).columns

cols = ['LTV', 'FrontDTI', 'borrowercount', 'TI','SSEC_Income','SEMP_Income', 'RNTL_Income',  
       'PNSN_Income', 'EMPL_Income','OTHR_Income' ]
impute = SimpleImputer(missing_values=np.nan,strategy='mean')
df[cols] = impute.fit_transform(df[cols])

#### Current LTV

In [14]:
#df['currentLTV']= df['upb']/df['Leadgen_Value']
#df['currentLTV'] = np.where(df['currentLTV'].isnull(), df['LTV'], df['currentLTV'])

##### Income Percent

In [15]:
# income source convert to percentage 
df['EMPL_Income_per'] = df['EMPL_Income']/df['TI']
df['OTHR_Income_per'] = df['OTHR_Income']/df['TI']
df['PNSN_Income_per'] = df['PNSN_Income']/df['TI']
df['RNTL_Income_per'] = df['RNTL_Income']/df['TI']
df['SEMP_Income_per'] = df['SEMP_Income']/df['TI']
df['SSEC_Income_per'] = df['SSEC_Income']/df['TI']
df['OTHR_Income_per'] = df['OTHR_Income']/df['TI']

#### Payment Related

In [16]:
# payment info
df['current_total_pmt'] = df['currentescrowpayment'] + df['currentpipayment']
#df['LastPaymentAmt'] = np.where(df['LastPaymentAmt'].isnull, 0, df['LastPaymentAmt'])
df['upb_change']= df['upb'] -df['priorupb']
#df['pmt_diff'] = df['current_total_pmt'] - df['LastPaymentAmt']
#df['upb_minus_pmt_diff']= df['upb_change'] - df['pmt_diff']

#### Categorical Features

In [82]:
# Check missing data counts FOR OBJECT 
#cols = df.select_dtypes(np.object).columns
#df[cols].isna().mean().round(4)*100

loanstatusid                    0.00
loanstatus                      0.00
priorstatus                     0.58
EscrowFlag                      0.00
ForebearanceStatusCodeId       93.26
InvestorId                      0.00
DraftingIndicator              78.68
InvestorName                    0.00
status24months                  0.58
CampaignProfile                 0.03
COVIDforbearancetype           94.74
producttype                     0.00
selfemployed                   25.85
DUtype                         24.08
RealEstateOwnedCount           37.38
LoanPurpose                     0.00
fsttimehomebuyer                3.66
amortizationtype                0.00
doctype                        24.08
PropState                       0.00
zipcode                         0.00
CBSACode                        0.00
PropertyType                    0.00
Occupancy                       0.00
channel                         0.00
HighBalanaceFlag                0.00
Entity                          0.00
t

#### Missing Imputation for Categorical

In [17]:
# using mode to impute missing categorical variables 
cols = ['fsttimehomebuyer']
impute = SimpleImputer(strategy="most_frequent") 
df[cols] = impute.fit_transform(df[cols])

#### Binary Flags (missing is more than 10%)

In [18]:
df['forebearance_status_flag']=np.where(df['ForebearanceStatusCodeId'].isnull(),0,1)
df['drafting_ind_flag']=np.where(df['DraftingIndicator'].isnull(),0,1)
df['Covid_flag']= np.where(df['COVIDforbearancetype'].isnull(), 0,1)
df['selfemployed_flag']= np.where(df['selfemployed'].isnull(), 0,1)
df['DUtype_flag']= np.where(df['DUtype'].isnull(), 0,1)
df['doctype_flag']= np.where(df['doctype'].isnull(), 0,1)
df['tract_income_category_flag']= np.where(df['tract_income_category'].isnull(), 0,1)
df['application_income_category_flag']= np.where(df['application_income_category'].isnull(), 0,1)


#extra
df['mod_flag'] = np.where(df['LoanModificationDt'].isnull(),0,1)
df['draft_flag']=np.where(df['DraftingIndicator'].isnull(),0,1)
df['ACHPmtSet_flag']= np.where(df['ACHPmtSet'].isnull(), 0,1)

#### RE Owned Count (RE owned count is mixed type)

In [19]:
#fill_na_mean(df['RealEstateOwnedCount'])
df['RealEstateOwnedCount'] = pd.to_numeric(df['RealEstateOwnedCount'], errors='coerce')
df['RealEstateOwnedCount'] = df['RealEstateOwnedCount'].fillna(value=df['RealEstateOwnedCount'].mean())

#### Lagged Variables

In [20]:
df['loanstatusid_lag1'] = df.groupby(['loanid','nextpaymentduedt'])['loanstatusid'].shift(1)
df['loanstatusid_lag1']= df['loanstatusid_lag1'].fillna(method='bfill') 

# create lagged payment variables 
df['lag1_PI'] = df.groupby(['loanid'])['currentpipayment'].shift(1)
df['lag1_Escrow'] = df.groupby(['loanid'])['currentescrowpayment'].shift(1)
df['lag1_total_pmt'] = df.groupby(['loanid'])['current_total_pmt'].shift(1)

#fill previous values 
df['lag1_PI']= df['lag1_PI'].fillna(method='bfill') 
df['lag1_Escrow']= df['lag1_Escrow'].fillna(method='bfill') 
df['lag1_total_pmt'] =df['lag1_total_pmt'].fillna(method='bfill') 

df['TI_change'] = df['current_total_pmt'] - df['lag1_total_pmt'] 
df['PI_change'] = df['currentpipayment'] - df['lag1_PI'] 
df['Escrow_change'] = df['currentescrowpayment'] - df['lag1_Escrow'] 

#### Avg Payment Days


In [21]:
avg_payment_day('AvgPmtDay_3M')
avg_payment_day('AvgPmtDay_6M')
avg_payment_day('AvgPmtDay_12M')

#### Streamline

In [22]:
df['streamline_flag']= np.where(df['streamlineflag']=='SL',1,0)

#### Check Missing %

In [ ]:
#df.isna().mean().round(4)*100

#### Outlier Treatments

In [1]:
##

#### Export Processed data back to S-3

In [23]:
%%time
import awswrangler as wr
wr.s3.to_parquet(
    df=df,
    path='s3://ppsg-collections/2021-06-01/consolidated/random_data/consolidated_sample_expanded_treated_v1.parquet'
)

CPU times: user 2min 57s, sys: 32.2 s, total: 3min 30s
Wall time: 3min 6s


{'paths': ['s3://ppsg-collections/2021-06-01/consolidated/random_data/consolidated_sample_expanded_treated_cltv.parquet'],
 'partitions_values': {}}

In [52]:
#check_missing = df.isna().mean().round(4)*100
#check_missing.sort_values()

#### Check Good and Bad Counts

In [99]:
# def create_good_bad_table(df):
#     table= pd.crosstab(index=df['PastDue'], columns=[df['Down2']]).reset_index()
#     table= table.rename(columns={0: "G", 1: "B"})
#     table['percent'] = table['B']/( table['G']+ table['B'])*100
#     return table

In [100]:
# score = df[(df["CampaignProfile"]== "Never Delinquent") & (df["rollback"]== 0) 
#            & ((df["nextpaymentduedt"] >='2020-02-01') & (df["nextpaymentduedt"] <='2021-04-01'))]
# create_good_bad_table(score)

Down2,PastDue,G,B,percent
0,1,2338954,35447,1.492882
1,2,2109763,35441,1.652104
2,3,1933601,35442,1.799961
3,4,1768863,35442,1.964302
4,5,1541965,35443,2.246914
5,6,1422695,35443,2.430703
6,7,1320170,35443,2.614537
7,8,1215360,35443,2.833620
8,9,1117327,35443,3.074594
9,10,965049,35443,3.542557


In [55]:
#df[(df['nextpaymentduedt']=='2020-08-01') & (df['loanid']==1000017603)][['rundt','PastDue']]